##### 1. Remove records with null customer id

In [0]:
%sql
SELECT * FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

##### 2. Remove duplicate records

In [0]:
%sql
SELECT DISTINCT * FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW v_distinct_customers
AS
SELECT DISTINCT * FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

###### 3. Remove duplicate records based on created timestamp

In [0]:
%sql
WITH max_customers AS (
  SELECT customer_id,
        max(created_timestamp) as max_timestamp
  FROM gizmobox.bronze.v_customers
  WHERE customer_id IS NOT NULL
  GROUP BY customer_id
)
SELECT c.* FROM v_distinct_customers c
INNER JOIN max_customers m
ON c.customer_id = m.customer_id
AND c.created_timestamp = m.max_timestamp;

##### 4. Cast column value to the correct data type

In [0]:
%sql
WITH max_customers AS (
  SELECT customer_id,
        max(created_timestamp) as max_timestamp
  FROM gizmobox.bronze.v_customers
  WHERE customer_id IS NOT NULL
  GROUP BY customer_id
)
SELECT CAST(c.created_timestamp as TIMESTAMP) AS created_timesatmp,
        c.customer_id,
        c.customer_name,
        CAST(c.date_of_birth AS DATE) AS date_of_birth,
        c.email,
        CAST(c.member_since AS DATE) AS member_since,
        c.telephone
         FROM v_distinct_customers c
INNER JOIN max_customers m
ON c.customer_id = m.customer_id
AND c.created_timestamp = m.max_timestamp;

##### 5. Write customer data to Delta table

In [0]:
%sql
CREATE TABLE gizmobox.silver.customers
AS 
WITH max_customers AS (
  SELECT customer_id,
        max(created_timestamp) as max_timestamp
  FROM gizmobox.bronze.v_customers
  WHERE customer_id IS NOT NULL
  GROUP BY customer_id
)
SELECT CAST(c.created_timestamp as TIMESTAMP) AS created_timesatmp,
        c.customer_id,
        c.customer_name,
        CAST(c.date_of_birth AS DATE) AS date_of_birth,
        c.email,
        CAST(c.member_since AS DATE) AS member_since,
        c.telephone
         FROM v_distinct_customers c
INNER JOIN max_customers m
ON c.customer_id = m.customer_id
AND c.created_timestamp = m.max_timestamp;

In [0]:
%sql
DESCRIBE EXTENDED gizmobox.silver.customers;